In [148]:
from re import findall, DOTALL, IGNORECASE, search
IVIeyeInfo='''s/p IVIA OS on 103-4-21
s/p phaco-IOL OD on 102-11-20
s/p IVIA OS on 102-10-16
s/p phaco-IOL OS 102-09-18
s/p IVIA OD on 102-4-3
s/p IVIA OS 102-2-6
s/p IVIA OS on 101-8-29
s/p IVIA OU on 101-02-22, 101-1-9, 100-11-23
s/p IVIL OU on 100-6-15, 100-5-11, 100-04-11 (approved NHI)'''
findall('(IVI[ALEO])\s+(O[SDU])\s+on\s+(.*)', IVIeyeInfo)


[('IVIA', 'OS', '103-4-21'),
 ('IVIA', 'OS', '102-10-16'),
 ('IVIA', 'OD', '102-4-3'),
 ('IVIA', 'OS', '101-8-29'),
 ('IVIA', 'OU', '101-02-22, 101-1-9, 100-11-23'),
 ('IVIL', 'OU', '100-6-15, 100-5-11, 100-04-11 (approved NHI)')]

In [235]:
from pandas import read_csv
df=read_csv('IVILEOA.csv', delimiter='\x06', dtype='str')
#df.outcome.head(10)[0]

In [66]:
smpIVI='''
S:主觀資料
s/p IVIA OS on 103-4-21
s/p phaco-IOL OD on 102-11-20
s/p IVIA OS on 102-10-16
s/p phaco-IOL OS 102-09-18
s/p IVIA OD on 102-4-3
s/p IVIA OS 102-2-6
s/p IVIA OS on 101-8-29
s/p IVIA OU on 101-02-22, 101-1-9, 100-11-23
s/p IVIL OU on 100-6-15, 100-5-11, 100-04-11 (approved NHI), OU on 100-6-15, 100-5-11, 100-04-11
s/p IVIA OU 1000126
s/p IVIA OS 991117
s/p PDT OU 990318
s/p IVIA OU 990317
s/p PDT OU 980730
s/p IVIA OU 980729
DM for 10 years, HTN+, CAD, proteinuria

'''
#print(smpIVI)
for IVIeyeInfo in findall('S:主觀資料\n(.*)\n\n', smpIVI, DOTALL):
    print(IVIeyeInfo)
    #eyeInfo=findall('(IVI[ALEO])\s+(O[SDU])\s+on\s+(.*)', IVIeyeInfo)
    eyeInfo=findall('(IVI[ALEO])\s+(O[SDU])\s+on\s+(.*),?\s+(O[SDU])\s+on(.*)', IVIeyeInfo)
    print(eyeInfo)

s/p IVIA OS on 103-4-21
s/p phaco-IOL OD on 102-11-20
s/p IVIA OS on 102-10-16
s/p phaco-IOL OS 102-09-18
s/p IVIA OD on 102-4-3
s/p IVIA OS 102-2-6
s/p IVIA OS on 101-8-29
s/p IVIA OU on 101-02-22, 101-1-9, 100-11-23
s/p IVIL OU on 100-6-15, 100-5-11, 100-04-11 (approved NHI), OU on 100-6-15, 100-5-11, 100-04-11
s/p IVIA OU 1000126
s/p IVIA OS 991117
s/p PDT OU 990318
s/p IVIA OU 990317
s/p PDT OU 980730
s/p IVIA OU 980729
DM for 10 years, HTN+, CAD, proteinuria
[('IVIL', 'OU', '100-6-15, 100-5-11, 100-04-11 (approved NHI),', 'OU', ' 100-6-15, 100-5-11, 100-04-11')]


In [237]:
def prprDate(vl):
    if vl.find('-')!=-1:pass
    if len(vl)==7: pass
    else: return 
def infoIVI(row):
    rowNdx, (hist, mrn1, mrn2, sdate, outcome)=row.name, row
    for IVIeyeInfo in findall('S:主觀資料\n(.*)\n\nO:', outcome, DOTALL):
        #if eyeInfo:=findall('(IVI[ALEO])\s+(O[SDU])\s+on\s+(.*),?(O[SDU])\s+on(.*)', IVIeyeInfo, IGNORECASE):
        if eyeInfo:=findall('(IVI[ALEO])(.*)', IVIeyeInfo, IGNORECASE):
            #print('ori eyeInfo', eyeInfo)
            for ivctgry, sdateInfo in eyeInfo:
                ivCtgry={}
                for osType, _, sDate in findall('(O[SDU]) (on )?(.*)', sdateInfo):
                    print('oriSdate', osType, sDate)
                    oType={}
                    otype=osType.upper()
                    if oriSdate:=oType.get(otype):
                        oriSdate='|'.join(findall('\d{4,8}', oriSdate))
                        fullSdate='|'.join([oriSdate,sdate])
                        oType[otype]=fullSdate
                    else:
                        sdate=sdate.replace('-', '0').replace('/', '0')
                        #sdate='|'.join(map(prprDate, sdate))
                        sdate='|'.join(findall('\d{4,8}', sdate))
                        oType[otype]=sdate
                    ivCtgry.update({ivctgry:oType})
            print(len(eyeInfo), eyeInfo, hist, ivCtgry)
fout=open('IVILOAE2.csv', 'w')
from json import dumps
def rtrvIVI(row):
    rowNdx, (hist, mrn1, mrn2, sdate, outcome)=row.name, row
    for IVIeyeInfo in findall('S:主觀資料\n(.*)\n\nO:', outcome, DOTALL):
        if eyeInfo:=findall('(IVI[ALEO]|PDT)(.*)', IVIeyeInfo, IGNORECASE):
            #print(eyeInfo)
            ivCtgry, oType={}, {}
            for ivctgry, sdateInfo in eyeInfo:
                for otype, _, infoDate in findall('(O[SDU]) (on )?(.*)', sdateInfo):
                    for year, month, day in findall('(\d+)[-/]?(\d+)[-/]?(\d+)', infoDate):#\d{5,8}year, month, day(\d{2,4})[-/]?(\d{1,2})[-/]?(\d{1,2})
                        #print(year, month, day)
                        dateInfo=f'{year}{month}{day}'
                        lenDate=len(dateInfo)
                        if lenDate==8: year, month, day=search('(\d{4})[-/]?(\d{2})[-/]?(\d{2})', dateInfo).groups()#.replace('-', '0').replace('/', '0')
                        elif lenDate==7: year, month, day=search('(\d{3})[-/]?(\d{2})[-/]?(\d{2})', dateInfo).groups()
                        elif lenDate==6: 
                            year, month, day=search('(\d{2})[-/]?(\d{2})[-/]?(\d{2})', dateInfo).groups()
                            if int(year)<=21:year=f'20{year}'
                            if int(month)>12: year, month, day=search('(\d{3})[-/]?(\d{1})[-/]?(\d{2})', dateInfo).groups()
                        elif lenDate==5:
                            year, month, day=search('(\d{3})[-/]?(\d)[-/]?(\d)', dateInfo).groups()
                            if int(year)>110: year, month, day=search('(\d{2})[-/]?(\d)[-/]?(\d\d)', dateInfo).groups()
                        elif lenDate==4:
                            year, month, day=search('(\d{2})[-/]?(\d)[-/]?(\d)', dateInfo).groups()
                            if int(year)<21: year=2010
                        
                        if len(str(year))<=3: year=int(year)+1911
                        if len(str(month))==1: month=month.zfill(2)
                        if len(str(day))==1: day=day.zfill(2)
                        #print(year, month, day)
                        Sdate=f'{year}{month}{day}'
                        if oriSdate:=oType.get(otype):
                            #oriSdate='|'.join(findall('\d{4,8}', oriSdate))
                            fullSdate='|'.join([oriSdate,Sdate])
                            oType[otype]=fullSdate
                        else:
                            #sdate='|'.join(map(prprDate, sdate))
                            #sdate='|'.join(findall('\d{4,8}', sdate))
                            if Sdate: oType[otype]=Sdate
                    if oType: ivCtgry.update({ivctgry:oType})
            #rint(len(eyeInfo), eyeInfo, hist, ivCtgry)
            if ivCtgry: fout.write(','.join([hist, dumps(ivCtgry)])+'\n')

df.apply(rtrvIVI, axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
202694    None
202695    None
202696    None
202697    None
202698    None
Length: 202699, dtype: object

In [185]:
#df.columns
findall('(\d\d[-/]?\d\d[-/]?\d\d)', 'OU 1# 1010615, 2# 1010720, 3# 1010831')#' OD 96-4-11; arrange VT+PRP, OD 96-7-9')

['101061', '101072', '101083']

In [186]:
#d1, d2={'a':'apple1'}, {'a':'apple2'}
'2'.zfill(2)


'02'

In [236]:
#findall('\d+', '104-0115')
df

chartid mrn1 mrn2       sdate  \
0       13745476  NaN  010  2015-01-27   
1       23103686  NaN  004  2012-02-07   
2       30375441  NaN  110  2019-03-19   
3         271090  NaN  010  2017-12-01   
4       37899459  NaN  110  2011-06-21   
...          ...  ...  ...         ...   
202694  13788153  NaN  182  2011-06-13   
202695  43961279  NaN  1PH  2017-12-07   
202696  31690387  NaN  010  2016-12-26   
202697  23240111  NaN  182  2016-07-11   
202698  25547880  NaN  010  2019-05-13   

                                                  outcome  
0       BP: 0 / 0 PP: 0 BH: 164 BW: 0\nS:主觀資料\ns/p IVI...  
1       BP: 0 / 0 PP: 0 BH: 161.3 BW: 0\nS:主觀資料\nHCV-H...  
2       BP: 0 / 0 PP: 0 BH: 0 BW: 0\nS:主觀資料\nTSCP OS o...  
3       BP: 0 / 0 PP: 0 BH: 175 BW: 0\nS:主觀資料\nCC: met...  
4       BP: 0 / 0 PP: 0 BH: 172 BW: 0\nS:主觀資料\nIVIA OD...  
...                                                   ...  
202694  BP: 0 / 0 PP: 0 BH: 0 BW: 0\nS:主觀資料\n50 Y/O,G4...  
202695  BP: 0 / 0 PP: 0 BH: 0 BW: 0\nS:主觀資料\nchoroidal...  
202696  BP: 0 / 0 PP: 0 BH: 158 BW: 0\nS:主觀資料\nfelt mo...  
202697  BP: 0 / 0 PP: 0 BH: 155 BW: 0\nS:主觀資料\n73Y/O,P...  
202698  BP: 0 / 0 PP: 0 BH: 154 BW: 0\nS:主觀資料\ns/p IVI...  

[202699 rows x 5 columns]

In [201]:
fout=open('/tmp/dummy.csv', 'w')
from json import dumps
#json.dumps(dict)
fout.write(dumps(d1))

15

In [230]:
from altair import  Chart
#alt.data_transformers.enable('data_server')  # handle larger datasets
from pandas import DataFrame
tseries={"OD": "20170315|20170510|20170809|20171129|20170215|20160810|20180221|20160601|20160713".split('|')}
#len(tseries)
#tseries = {'col_1': [3, 2, 1, 0], 'col_2': ['a', 'b', 'c', 'd']}
df=DataFrame.from_dict(tseries, dtype='str')
Chart(df).mark_line().transform_fold( ['OD', ]).encode( x='hours(Date):T', y='sum(value):Q')#,  color='key:OD')

alt.Chart(...)

In [251]:

[x for x in findall('(\w+\.?/?\d?\.?\d?)\+?.*?', 'VA: 6/4+0.25-0.75x73, 6/60+1.25-1.50x88, IOP: 15/14 mmHg;') if x.find('/')!=-1 ]#  else None]#, IGNORECASE) rtrvODOS


['6/4', '6/60', '15/14']

In [264]:
findall('(CF|NLP|LP|HM|\d+\.?/?\d?\.?\d?)c?c?[(+-]?.*?', '6/10cc(+2.75-1.5x80) OD, 6/60cc(+4.0-1.0x90) OS, IOP 18/21 mmHg', IGNORECASE)# 'VA: 0.16-2.0-2.75x40 OD, 6/8.6 -0.75-2.25X135cc OS')

['6/10', '2.75', '1.5', '80', '6/60', '4.0', '1.0', '90', '18/21']

In [ ]:
A: CF OD, 5/60cc OS, IOP 15/15 mmHg   
     

In [282]:
objInfo='''O:客觀資料
VA: 6/6cc OD, 6/7.5cc OS,
Ref: -4.5-0.75x123/-4.5-0.25x174'''
#NCTIOP= 15/14 mmHg'''
findall('O:客觀資料\n(.*?)\n(.*IOP.*mmHg)?', abcInfo, DOTALL)

[]

In [300]:
abcInfo='''O:客觀資料
VA:6/6.7cc OD 6/7.5cc OS 
IOP: 16/16 mmHg 
'''
#VA OD 6/30cc, OS 1/60cc, IOP 15/16 mmHg'''

In [301]:
#          |            | VA:OD 6/8.6cc,OS LP? IOP: 14/31(N)11/30(G)mmHg                                  +|                                             | 
findall('O:客觀資料\n(.*)[\n]?(.*IOP.*mmHg)?\n', abcInfo, DOTALL)

[('VA:6/6.7cc OD 6/7.5cc OS \nIOP: 16/16 mmHg ', '')]

In [304]:
findall('O:客觀資料\n(.*OD.*OS.*?)[\n]+(.*IOP.*mmHg)?', abcInfo, DOTALL)

[('VA:6/6.7cc OD 6/7.5cc OS ', 'IOP: 16/16 mmHg')]

In [345]:
defINFO='''O:客觀資料
VA: 6/10cc OD, 0.16cc OS,
1080417 AR +0.25-1.25x161/-0.5-1.0x6
1080102 AR -1.5x169/-5.25-0.75x175
1071114 AR -0.25-1.5x165/-5.75-0.5x180
1070912 AR +0.00-2.25x170/-5.50-1.00x5
1070822 AR: -0.25-1.5x1/-5.5-1.0x2
Ref:-6.00-2.75*168/-5.75-1.00*179
NCTIOP= 16/18mmHg
'''
findall('O:客觀資料\n(.*OD.*OS.*?\n?).*?\n?(.*IOP.*mmHg)?', fghInfo, DOTALL)

[('VA: 6/12 cc OD, 6/8.6 cc OS\n', 'IOP: 17/18mmHg')]

In [338]:
fghInfo='''
O:客觀資料
VA: 6/12 cc OD, 6/8.6 cc OS IOP: 17/18mmHg
'''

In [340]:
'O:客觀資料\n(.*OD.*OS.*?\n?).*?\n?(.*IOP.*mmHg)?'=='O:客觀資料\n(.*OD.*OS.*?\n?).*?\n?(.*IOP.*mmHg)?'

True

In [344]:
findall('IOP.*?(\d+/\d+)','\n1080417 AR +0.25-1.25x161/-0.5-1.0x6\n1080102 AR -1.5x169/-5.25-0.75x175\n1071114 AR -0.25-1.5x165/-5.75-0.5x180\n1070912 AR +0.00-2.25x170/-5.50-1.00x5\n1070822 AR: -0.25-1.5x1/-5.5-1.0x2\nRef:-6.00-2.75*168/-5.75-1.00*179\nNCTIOP= 16/18mmHg')

['16/18']

In [350]:
for 客觀資料, IOPinfo in findall('O:客觀資料\n(.*OD.*OS.*?\n?).*?\n?(.*IOP.*mmHg)?', fghInfo, DOTALL):  #'O:客觀資料\n(.*?IOP.*?)\n' 'O:客觀資料\n(.*?)[\n]?(.*IOP.*mmHg)?\n' #O:客觀資料\n(.*)[\n]?(.*IOP.*mmHg)?
  mtchIOP=search('\d+/\d+', IOPinfo)
  if mtchIOP: spcfcIOP=mtchIOP.group()
  print(spcfcIOP, findall('(CF|NLP|LP|HM|\d+\.?/?\d?\.?\d?)[+-]?.*?', 客觀資料) )
dmyInfo='''
ntRslt=[x for x in findall('(CF|NLP|LP|HM|\d+\.?/?\d?\.?\d?)[+-]?.*?', line) ]#if x.find('/')!=-1 ]
lenRSLT=len(ntRslt)
if lenRSLT>3:
  cmbPttrnRlst=findall(rightPttrn+hgPttrn, outcome)
  if cmbPttrnRlst:
    #print('cmbRIGHT', outcome, cmbPttrnRlst)
    sghtInfo='|'.join(cmbPttrnRlst[0])
  cmbPttrnRlst=findall(pttrnLEFT+hgPttrn, outcome)
  if cmbPttrnRlst:
    sghtInfo='|'.join(cmbPttrnRlst[0])
    #print('cmbLEFT', outcome, cmbPttrnRlst)
  cmbPttrnRlst=findall(pttrnLEFT, outcome)
  if cmbPttrnRlst:
    sghtInfo='|'.join(cmbPttrnRlst[0])
    #print('LEFTcmb', outcome, cmbPttrnRlst)
  else:
    #print('rightPttrn', outcome, 
    pttrnRslt=findall(f'O[DS]:?\s?{dcmlPttrn}', outcome, IGNORECASE)
    if pttrnRslt: sghtInfo='|'.join(pttrnRslt)
elif lenRSLT==3:
  #print('3rslt od, os, mmHg', ntRslt)
  sghtInfo='|'.join(ntRslt)
  #print(f'{outcome}==>', '|'.join(ntRslt))
elif lenRSLT==2:
  sghtInfo='|'.join(ntRslt)
  #print('2rslt od, os', ntRslt)
if sghtInfo and search('\d{6,}', sghtInfo): sghtInfo=None
'''


17/18 ['6/12', '6/8.6']


In [347]:
ntRslt

[]

In [353]:
[1, 3, 4, 4][:2]

[1, 3]

In [378]:
findall('O:客觀資料\n(.*OD.*OS.*\n?).*?\n?(.*?IOP.*mmHg)?', 'O:客觀資料\nOD 6/15cc, OS 6/8.6cc, IOP 16/15 mmHg', DOTALL)

[('OD 6/15cc, OS 6/8.6cc, IOP 16/15 mmHg', '')]

In [382]:
findall('O:客觀資料\n(.*)\n', 'O:客觀資料\nVA OD 6/6cc, OS 6/6cc  IOP 18/20mmHg\n', DOTALL|IGNORECASE)

['VA OD 6/6cc, OS 6/6cc  IOP 18/20mmHg']

In [383]:
findall('(CF|NLP|LP|HM|\d+\.?/\d?\.?\d?)[+-]?.*?', 'VA OD 6/6cc, OS 6/6cc  IOP 18/20mmHg')

['6/6', '6/6', '18/20']

In [386]:
dmyInfo='''O:客觀資料
VA: 6/20cc OD, LP OS;
pneumotonometer: 12/34 mmHg;
Cornea: Microcystic edema OS,
A/C: no hyphema os
Iris: no NVI os, Pupil: synechia os
slitlamp: decreased taer meniscus OU congested conj ou, lens: NS++~+++ OD,
IOL in position OS
96-11-15VF: central island OD
FAG 981203: macular atrophy OS>OD
C/D: 0.85-0.85 od 0.95-0.95os
980824 oct: MT 199/536um
Fd 990318: CRVO OS
Fd 990707: macular c retinl H os, CRVO os,
Fd 990929: pale disc os,
Fd 1000105: not clear seen os
Fd 100-6-22: Pale disc OS.'''
客觀資料=findall('O:客觀資料\n(.*?)\n', dmyInfo, DOTALL)[0]

In [387]:
客觀資料

'VA: 6/20cc OD, LP OS;'

In [390]:
findall('(CF|NLP|LP|HM|\d+\.?/?\d?\.?\d?)', 'VA OD 4/60cc OS HM/15cm, IOP:12/12 mmHg,')

['4/60', 'HM', '15', '12/12']

In [398]:
[x+y for x, y in findall('(CF|NLP|LP|HM|\d+\.?)(/?\d?\.?\d?[+-]?.*?)', 'VA OD 4/60cc OS HM/15cm, IOP:12/12 mmHg,')]

['4/60', 'HM/15', '12/12']

In [448]:
dmyInfo2='''O:客觀資料
OD (6/5c-5.25-1.25x70), OS 6/7.5 -5.25 -0.75x130
silent OU, lens: mild opacity on nucleus OU, Ind ophthalmoscopy: mac
IOP:19/17mmHg, mild congested conj OU, slitlamp: clear cornea OU, A/C deep

'''
#ODOS, IOPinfo=findall('O:客觀資料\n(.*OD.*OS.*?\n?).*?\n?(.*IOP.*mmHg)?', dmyInfo2, DOTALL|IGNORECASE)[0]


['OD (6/5c-5.25-1.25x70), OS 6/7.5 -5.25 -0.75x130\nsilent OU, lens: mild opacity on nucleus OU, Ind ophthalmoscopy: mac\nIOP:19/17mmHg, mild congested conj OU, slitlamp: clear cornea OU, A/C deep']

In [453]:
#rsltSght
#ODOSinfo, IOPinfo=
findall('(.*OD.*OS.*?\n).*?\n?(.*IOP.*mmHg)?', 'VA:OD HM ,OS 6/30cgl\nIOP 15/15 mmHg\nSlitlamp: Conj.congestion OU,cornea SPKs OD AC deep,clear OU,', DOTALL|IGNORECASE)[0]  #'O:客觀資料\n(.*OD.*OS.*\n?).*?\n?(.*?IOP.*mmHg)?'
#findall('(\d+/\d+)', ODOS, DOTALL)

('VA:OD HM ,OS 6/30cgl\n', 'IOP 15/15 mmHg')

In [456]:
#IOPinfo, ODOS
findall('(CF|NLP|LP|HM|\d+\.?)(/?\d?\.?\d?[+-]?.*?)', 'VA:OD HM ,OS 6/30cgl\nIOP 15/15 mmHg\nS') #'VA:OD HM ,OS 6/30cgl\nIOP 15/15 mmHg\nSlitlamp')

[('HM', ''), ('6', '/30'), ('15', '/15')]

In [429]:
#ODOSinfo, IOPinfo=
findall('O:客觀資料\n(.*OD.*OS.*?\n?).*?\n?(.*IOP.*mmHg)?', dmyInfo2, DOTALL|IGNORECASE)  #'O:客觀資料\n(.*OD.*OS.*\n?).*?\n?(.*?IOP.*mmHg)?'


[('OD (6/5c-5.25-1.25x70), OS 6/7.5 -5.25 -0.75x130\nsilent OU, lens: mild opacity on nucleus OU, Ind ophthalmos',
  'copy: mac\nIOP:19/17mmHg')]

In [432]:
findall('\d+/\d+', 'OD (6/5c-5.25-1.25x70), OS 6/7.5 -5.25 -0.75x130\nsilent OU, lens: mild opacity on nucleus OU, Ind ophthalmos')

['6/5', '6/7']

In [433]:
findall('\d+/\d+', 'copy: mac\nIOP:19/17mmHg')

['19/17']

In [435]:
findall('(CF|NLP|LP|HM|\d+\.?)(/?\d?\.?\d?[+-]?.*?)', '(104-1-20) 6/10cc, 6/10cc, IOP 22/13mmHg, AR +1.50-2.25x80/ +1.0-1.50x105')

[('104', '-'),
 ('1', '-'),
 ('20', ''),
 ('6', '/10'),
 ('6', '/10'),
 ('22', '/13'),
 ('1.', '50-'),
 ('2.', '25'),
 ('80', '/'),
 ('1.', '0-'),
 ('1.', '50'),
 ('105', '')]

In [437]:
search('/\d','/13')

<re.Match object; span=(0, 2), match='/1'>

In [439]:
findall('(CF|NLP|LP|HM|\d+\.?)(/?\d?\.?\d?[+-]?.*?)', '客觀資料\nVa: OD CF/5cm, OS 5/60cc(+0.25-2.00x80);\nIOP: 14/11mmHg\n', DOTALL)

[('CF', '/5'),
 ('5', '/60'),
 ('0.', '25-'),
 ('2.', '00'),
 ('80', ''),
 ('14', '/11')]

In [440]:
mtchODOSIOP=[x+y for x, y in findall('(CF|NLP|LP|HM|\d+\.?)(/?\d?\.?\d?[+-]?.*?)', '客觀資料\nVa: OD CF/5cm, OS 5/60cc(+0.25-2.00x80);\nIOP: 14/11mmHg\n', DOTALL) if search('/\d', y)]


In [441]:
mtchODOSIOP

['CF/5', '5/60', '14/11']

In [442]:
len(mtchODOSIOP)

3

In [443]:
['CF/5', '5/60', '14/11', '297/29', '5/8', '2015/08'][:3]

['CF/5', '5/60', '14/11']

In [444]:
['CF/5', '5/60', '14/11'][:3]

['CF/5', '5/60', '14/11']

In [458]:
dtpInfo='''	     o := '11'; soapname := 'BP: '; 
		 dtsoaptx := dtsoap;
      WHEN 'DP' THEN 
	     o := '12'; soapname := ' / '; 
		 dtsoaptx := dtsoap;
      WHEN 'PP' THEN 
	     o := '13'; soapname := ' PP: '; 
		 dtsoaptx := dtsoap;
      WHEN 'BH' THEN 
	     o := '14'; soapname := ' BH: '; 
		 dtsoaptx := dtsoap;
      WHEN 'BW' THEN 
	     o := '15'; soapname := ' BW: '; 
		 dtsoaptx := dtsoap;
      WHEN 'HP' THEN 
	     o := '16'; soapname := ' HP: '; 
		 dtsoaptx := dtsoap;
      WHEN 'S'  THEN 
	     o := '21'; soapname := CHR(10) || 'S:主觀資料' || CHR(10);
		 dtsoaptx := dtsoap || CHR(10);
      WHEN 'O'  THEN 
	     o := '22'; soapname := CHR(10) || 'O:客觀資料' || CHR(10);
		 dtsoaptx := dtsoap || CHR(10);
      WHEN 'A'  THEN 
	     o := '23'; soapname := CHR(10) || 'A:診斷' || CHR(10);
		 dtsoaptx := dtsoap || CHR(10);
      WHEN 'T'  THEN 
	     o := '24'; soapname := CHR(10) || ' ' || CHR(10);
		 dtsoaptx := dtsoap || CHR(10);
      WHEN 'P'  THEN 
	     o := '25'; soapname := CHR(10) || 'P:治療計畫' || CHR(10);'''
findall("o := '(\d+)'; soapname :=(.*);", dtpInfo)

[('11', " 'BP: '"),
 ('12', " ' / '"),
 ('13', " ' PP: '"),
 ('14', " ' BH: '"),
 ('15', " ' BW: '"),
 ('16', " ' HP: '"),
 ('21', " CHR(10) || 'S:主觀資料' || CHR(10)"),
 ('22', " CHR(10) || 'O:客觀資料' || CHR(10)"),
 ('23', " CHR(10) || 'A:診斷' || CHR(10)"),
 ('24', " CHR(10) || ' ' || CHR(10)"),
 ('25', " CHR(10) || 'P:治療計畫' || CHR(10)")]

In [460]:
zippedInfo=[('11', " 'BP: '"),
 ('12', " ' / '"),
 ('13', " ' PP: '"),
 ('14', " ' BH: '"),
 ('15', " ' BW: '"),
 ('16', " ' HP: '"),
 ('21', " \n'S:主觀資料'\n"),
 ('22', " \n'O:客觀資料'\n"),
 ('23', " \n'A:診斷'\n"),
 ('24', " \n' '\n"),
 ('25', " \n'P:治療計畫'\n")]
dict(zippedInfo)

{'11': " 'BP: '",
 '12': " ' / '",
 '13': " ' PP: '",
 '14': " ' BH: '",
 '15': " ' BW: '",
 '16': " ' HP: '",
 '21': " \n'S:主觀資料'\n",
 '22': " \n'O:客觀資料'\n",
 '23': " \n'A:診斷'\n",
 '24': " \n' '\n",
 '25': " \n'P:治療計畫'\n"}

In [461]:
'OPH'=='OPH'

True

In [462]:
import asyncio

In [463]:
help(asyncio.wait)

Help on function wait in module asyncio.tasks:

async wait(fs, *, timeout=None, return_when='ALL_COMPLETED')
    Wait for the Futures and coroutines given by fs to complete.
    
    The fs iterable must not be empty.
    
    Coroutines will be wrapped in Tasks.
    
    Returns two sets of Future: (done, pending).
    
    Usage:
    
        done, pending = await asyncio.wait(fs)
    
    Note: This does not raise TimeoutError! Futures that aren't done
    when the timeout occurs are returned in the second set.



In [465]:
六祖壇經列表='''https://youtu.be/gafGM1CcGVY
https://youtu.be/djHFcp2pU1Y
https://youtu.be/gafGM1CcGVY
直播：https://youtu.be/djHFcp2pU1Y
https://youtu.be/Dpc0Pnf9Saw
https://youtu.be/F4KR7RN0Pyg
https://youtu.be/iQTSOFmE8lY
https://youtu.be/ZIIkuNdL1vQ
https://youtu.be/tmeM5hxpP50
https://youtu.be/a9YHI2b_uqw
https://youtu.be/4qcilg_Fqi4
https://youtu.be/spHIECuCAv0
https://youtu.be/7zger8StL9M
https://m.xuite.net/blog/ddrink910412/wretch/93079655
https://youtu.be/U89wQCbmDk0
https://youtu.be/4mtHlTTWOVQ
https://youtu.be/qn4divI5Xds
https://youtu.be/9lHF46nuoCg
https://youtu.be/NEHcYQogVH0
https://book.bfnn.org/books/0096.htm
https://youtu.be/Rs1aMdLeBDw
http://www.youtube.com/watch?v=AcgedHmj32o
https://kknews.cc/fo/q5arr5b.html
https://youtu.be/kxsM8QdLZJ4
https://youtu.be/_BBNTKacUVg
https://drive.google.com/drive/folders/1qIXblAl4riIXPIgo4mekrmEEey4EI0J9?usp=sharing
http://blog.udn.com/mobile/yea6969/5324298
https://zh.m.wikipedia.org/zh-tw/%E7%B6%AD%E6%91%A9%E8%A9%B0%E8%8F%A9%E8%96%A9
https://www.worldofmastermind.com/?p=7690
https://zh.m.wikipedia.org/zh-tw/%E7%B6%AD%E6%91%A9%E8%A9%B0%E8%8F%A9%E8%96%A9
https://www.worldofmastermind.com/?p=7690
https://youtu.be/Buv-TqS7McY
https://youtu.be/0cXTyTmDRUI
'''
壇經=set()
[壇經.add(vid) for vid in findall('https://youtu.be/(.*)', 六祖壇經列表)]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [2]:
for v in map(lambda x:x.zfill(2), '102/3/21'.split('/')):print(v)
''.join(map(lambda x:x.zfill(2), '102/3/21'.split('/')))

102
03
21


'1020321'

In [467]:
from os import system
from streamlit import altair_chart
for vid in 壇經:system(f'youtube-dl "https://youtu.be/{vid}"')

[youtube] spHIECuCAv0: Downloading webpage
[download] Destination: 懺悔歌-spHIECuCAv0.f244.webm
[download] 100% of 4.05MiB in 00:5535KiB/s ETA 00:00
[download] Destination: 懺悔歌-spHIECuCAv0.f251.webm
[download] 100% of 4.60MiB in 01:0050KiB/s ETA 00:00
[ffmpeg] Merging formats into "懺悔歌-spHIECuCAv0.webm"
Deleting original file 懺悔歌-spHIECuCAv0.f244.webm (pass -k to keep)
Deleting original file 懺悔歌-spHIECuCAv0.f251.webm (pass -k to keep)
[youtube] 9lHF46nuoCg: Downloading webpage
[youtube] 9lHF46nuoCg: Downloading player 10df06bb
[download] Destination: 2021 感懷師尊 35'26-9lHF46nuoCg.f137.mp4
[download] 100% of 497.82MiB in 02:46:42iB/s ETA 00:003007
[download] Destination: 2021 感懷師尊 35'26-9lHF46nuoCg.f140.m4a
[download] 100% of 32.82MiB in 10:5753KiB/s ETA 00:00:01
[ffmpeg] Merging formats into "2021 感懷師尊 35'26-9lHF46nuoCg.mp4"
Deleting original file 2021 感懷師尊 35'26-9lHF46nuoCg.f137.mp4 (pass -k to keep)
Deleting original file 2021 感懷師尊 35'26-9lHF46nuoCg.f140.m4a (pass -k to keep)
[youtube] dj

[download] Destination: 六祖慧能傳。電影(全)-gafGM1CcGVY.f242.webm
[download] 100% of 99.43MiB in 24:0709KiB/s ETA 00:00048
[download] Destination: 六祖慧能傳。電影(全)-gafGM1CcGVY.f140.m4a
[download] 100% of 86.18MiB in 20:3766KiB/s ETA 00:00120
[ffmpeg] Merging formats into "六祖慧能傳。電影(全)-gafGM1CcGVY.mkv"
Deleting original file 六祖慧能傳。電影(全)-gafGM1CcGVY.f242.webm (pass -k to keep)
Deleting original file 六祖慧能傳。電影(全)-gafGM1CcGVY.f140.m4a (pass -k to keep)
[youtube] _BBNTKacUVg: Downloading webpage
[youtube] _BBNTKacUVg: Downloading MPD manifest


[dashsegments] Total fragments: 1565
[download] Destination: 第十五講 壇經的殊勝與啟示-_BBNTKacUVg.f247.webm
[download]   9.7% of ~86.38MiB at 779.69KiB/s ETA 27:04463:58

ERROR: unable to download video data: The read operation timed out


[youtube] a9YHI2b_uqw: Downloading webpage
[youtube] a9YHI2b_uqw: Downloading MPD manifest


...

[dashsegments] Total fragments: 1654
[download] Destination: 第十一講   護法品第九-4mtHlTTWOVQ.f247.webm
[download]  42.4% of ~297.30MiB at 905.33KiB/s ETA 20:1212342[download] Got server HTTP error: HTTP Error 404: Not Found. Retrying fragment 702 (attempt 1 of 10)...
[download]  42.5% of ~297.34MiB at  1.00MiB/s ETA 20:102[download] Got server HTTP error: HTTP Error 404: Not Found. Retrying fragment 704 (attempt 1 of 10)...
[download]  42.7% of ~297.31MiB at 983.86KiB/s ETA 20:06[download] Got server HTTP error: HTTP Error 404: Not Found. Retrying fragment 708 (attempt 1 of 10)...
[download]  69.9% of ~301.62MiB at 13.23MiB/s ETA 10:3929[download] Got server HTTP error: HTTP Error 404: Not Found. Retrying fragment 1157 (attempt 1 of 10)...
[download]  71.5% of ~302.81MiB at 1022.80KiB/s ETA 10:04[download] Got server HTTP error: HTTP Error 404: Not Found. Retrying fragment 1184 (attempt 1 of 10)...
[download]  76.2% of ~302.74MiB at  1.13MiB/s ETA 08:2454[download] Got server HTTP error: HTTP

[download] Destination: 醒來-Buv-TqS7McY.f135.mp4
[download] 100% of 24.14MiB in 05:1079KiB/s ETA 00:006
[download] Destination: 醒來-Buv-TqS7McY.f251.webm
[download] 100% of 6.18MiB in 01:1979KiB/s ETA 00:000
[ffmpeg] Merging formats into "醒來-Buv-TqS7McY.mkv"
Deleting original file 醒來-Buv-TqS7McY.f135.mp4 (pass -k to keep)
Deleting original file 醒來-Buv-TqS7McY.f251.webm (pass -k to keep)
[youtube] 7zger8StL9M: Downloading webpage
[youtube] 7zger8StL9M: Downloading MPD manifest
[dashsegments] Total fragments: 62
[download] Destination: 懺悔歌-7zger8StL9M.f137.mp4


ERROR: unable to download video data: <urlopen error [Errno 101] Network is unreachable>


[youtube] F4KR7RN0Pyg: Downloading webpage


[download] Destination: 六祖壇經線上讀書會 第三講 （20210713）-F4KR7RN0Pyg.f136.mp4
[download] 100% of 274.66MiB in 57:5753KiB/s ETA 00:00919
[download] Destination: 六祖壇經線上讀書會 第三講 （20210713）-F4KR7RN0Pyg.f251.webm
[download] 100% of 130.44MiB in 27:3044KiB/s ETA 00:00707wn ETA
[ffmpeg] Merging formats into "六祖壇經線上讀書會 第三講 （20210713）-F4KR7RN0Pyg.mkv"
Deleting original file 六祖壇經線上讀書會 第三講 （20210713）-F4KR7RN0Pyg.f136.mp4 (pass -k to keep)
Deleting original file 六祖壇經線上讀書會 第三講 （20210713）-F4KR7RN0Pyg.f251.webm (pass -k to keep)


In [468]:
for vid in 壇經:print(f'"https://youtu.be/{vid}"')

"https://youtu.be/spHIECuCAv0"
"https://youtu.be/9lHF46nuoCg"
"https://youtu.be/djHFcp2pU1Y"
"https://youtu.be/gafGM1CcGVY"
"https://youtu.be/_BBNTKacUVg"
"https://youtu.be/a9YHI2b_uqw"
"https://youtu.be/Rs1aMdLeBDw"
"https://youtu.be/NEHcYQogVH0"
"https://youtu.be/4qcilg_Fqi4"
"https://youtu.be/iQTSOFmE8lY"
"https://youtu.be/0cXTyTmDRUI"
"https://youtu.be/U89wQCbmDk0"
"https://youtu.be/ZIIkuNdL1vQ"
"https://youtu.be/Dpc0Pnf9Saw"
"https://youtu.be/tmeM5hxpP50"
"https://youtu.be/qn4divI5Xds"
"https://youtu.be/kxsM8QdLZJ4"
"https://youtu.be/4mtHlTTWOVQ"
"https://youtu.be/Buv-TqS7McY"
"https://youtu.be/7zger8StL9M"
"https://youtu.be/F4KR7RN0Pyg"


In [3]:
a=[1, 2].extend([3])

In [4]:
data = df.reset_index().melt('x')